In [1]:
%pip install pandas beautifulsoup4 neo4j graphdatascience langchain jupyter


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [37]:
import os
import json
import pandas as pd 
from bs4 import BeautifulSoup, NavigableString, Tag
from urllib.parse import urlparse, urljoin
from graphdatascience import GraphDataScience
from neo4j import GraphDatabase

BASE_URL = 'https://muarwi.sharepoint.com/'
DATA_DIR = 'data'

In [ ]:
username = 'neo4j'
password = ''
uri = ''

## Test connection

In [39]:
with GraphDatabase.driver(uri, auth=(username, password)) as driver:
    driver.verify_connectivity()
    print('connection established.')

connection established.


## Helpers

In [40]:
class CustomBeautifulSoup(BeautifulSoup):
    """ 
    Customize `get_text()` method to include URL link in <a> tag.
    """
    def get_text(self, separator='', strip=False, types=(NavigableString,)):
        text_parts = []

        for element in self.descendants:
            if isinstance(element, NavigableString):
                text_parts.append(str(element))
            elif isinstance(element, Tag):
                if element.name == 'a' and 'href' in element.attrs:
                    text_parts.append(element.get_text(separator=separator, strip=strip))
                    text_parts.append(f'{element["href"]}')
            elif isinstance(element, types):
                text_parts.append(element.get_text(separator=separator, strip=strip))
        
        return separator.join(text_parts)


In [41]:
def get_uri_content_type(uri):
    """
    Get object type from a URL
    """
    # TODO
    if uri.startswith('mailto:'):
        return 'Email'
    elif uri.lower().endswith(('.pdf', '.doc', '.docx', '.xls', '.xlsx', '.ppt', '.pptx')):
        return 'Document'
    else:
        return 'WebPage'

In [42]:
def convert_relative_url(url):
    """
    Check the URL: if it is a relative URL, append BASE_URL.
    """
    if urlparse(url).netloc:
        return url
    else:
        return urljoin(BASE_URL, url)

In [43]:
def get_composite_id(webpart_id, page_id):
    """ 
    Create a composite ID for a webpart in a page. We need it because the webpart ID is not unique across pages.
    """
    return f'{page_id}#{webpart_id}'

## Methods to parse specific webparts

### `textWebPart`

In [44]:
def get_textWebPart(web_part, page_id)->object:
    """ 
    Get `textWebPart`

    Output structure:
    {
        'web_part': 'text_web_part',
        'text': <inner text from HTML>,
        'linked_to': [
            {
                type: 'page|email|docs|video|image|...',
                uri: <uri>
            },
            ...
        ]
    }
    """
    custom_bs = CustomBeautifulSoup(web_part['innerHtml'], 'html.parser')
    text = custom_bs.get_text('\n', strip=True)
    links = [convert_relative_url(a['href']) 
             for a in custom_bs.find_all('a', href=True)]
    twp = {
        'web_part': 'text_web_part',
        'id': get_composite_id(web_part['id'], page_id),
        'content': 'Text',
        'text': text,
        'linked_to': [{'type': get_uri_content_type(i), 'uri': i} for i in links]
    }
    return twp

### `StandardWebPart`

#### Call-to-action

In [45]:
def get_text_from_callToAction(web_part)->str:
    """Process webPart callToAction (id `df8e44e7-edd5-46d5-90da-aca1539313b8`)"""
    text = []
    processed_content = web_part['data']['serverProcessedContent']
    # overlay text
    overlay_text = next(item['value'] for item in processed_content['searchablePlainTexts'] if item['key'] == 'overlayText.text')
    text.append(f'Overlay text: {overlay_text}')
    # Link
    link = next(convert_relative_url(item['value']) for item in processed_content['links'] if item['key'] == 'button.linkUrl')
    text.append(f'Link: {link}')

    return '\n'.join(text)


def get_swp_call_to_action(web_part, page_id):
    '''
    df8e44e7-edd5-46d5-90da-aca1539313b8
    '''
    text = get_text_from_callToAction(web_part)
    processed_content = web_part['data']['serverProcessedContent']
    url = next(convert_relative_url(item['value']) for item in processed_content['links'] if item['key'] == 'button.linkUrl')
    cta = {
        'web_part': 'standard_web_part',
        'id': get_composite_id(web_part['id'], page_id),
        'content': 'CallToAction',
        'text': text,
        'linked_to': [{'type': get_uri_content_type(url), 'uri': url}]
    }

    return cta

#### File and Media

In [46]:
def get_text_from_fileAndMedia(web_part)->str:
    """Process webpart fileAndMedia (id `b7dd04e1-19ce-4b24-9132-b60a1c2b910d`)"""
    text = []
    processed_content = web_part['data']['serverProcessedContent']
    # text
    text += [f'{item["key"]}: {item["value"]}' for item in processed_content['searchablePlainTexts']]
    # relative link
    text += [f'{item["key"]}: {convert_relative_url(item["value"])}' 
             for item in processed_content['links'] if item['key'] == 'serverRelativeUrl']

    return '\n'.join(text)


def get_swp_file_and_media(web_part, page_id):
    ''' 
    Only contains 1 link to a file or media.
    '''
    text = get_text_from_fileAndMedia(web_part)
    url = convert_relative_url(web_part['data']['properties']['file'])
    fam = {
        'web_part': 'standard_web_part',
        'id': get_composite_id(web_part['id'], page_id),
        'content': 'FileAndMedia',
        'text': text,
        'linked_to': [{'type': get_uri_content_type(url), 'uri': url}]
    }
    return fam

#### People

In [47]:
def get_text_from_people(web_part)->str:
    """
    Process webpart with type `7f718435-ee4d-431c-bdbf-9c4ff326f46e`
    """
    processed_content = web_part['data']['serverProcessedContent']
    persons = web_part['data']['properties']['persons']
    spt = processed_content['searchablePlainTexts']

    text = []

    # title
    text.append(next(item['value'] for item in processed_content['searchablePlainTexts'] if item['key'] == 'title'))
    # Augment persons data
    for idx, person in enumerate(persons):
        person['name'] = next(item['value'] for item in spt if item['key'] == f'persons[{str(idx)}].name')
        person['email'] = next(item['value'] for item in spt if item['key'] == f'persons[{idx}].email')
 
    text += [f'name: {person["name"]}, role: {person["role"]}, email: {person["email"]}' for person in persons]

    return '\n'.join(text)


def get_swp_people(web_part, page_id):
    '''
    7f718435-ee4d-431c-bdbf-9c4ff326f46e 
    '''
    text = get_text_from_people(web_part)
    persons = web_part['data']['properties']['persons']
    spt = web_part['data']['serverProcessedContent']['searchablePlainTexts']

    people = {
        'web_part': 'standard_web_part',
        'id': get_composite_id(web_part['id'], page_id),
        'content': 'People',
        'text': text,
        'linked_to': [{
                        'id': person['id'],
                        'name': next(item['value'] for item in spt if item['key'] == f'persons[{str(idx)}].name'),
                        'email': next(item['value'] for item in spt if item['key'] == f'persons[{idx}].email'),
                        'role': person['role']
                    } for idx, person in enumerate(persons)]
    }

    return people

#### Image

In [48]:
def get_text_from_image(web_part)->str:
    """
    Process webpart with type `d1d91016-032f-456d-98a4-721247c305e8`
    """
    text = []
    processed_content = web_part['data']['serverProcessedContent']

    for idx in range(len(processed_content['imageSources'])):
        if processed_content['searchablePlainTexts']:
            caption = processed_content['searchablePlainTexts'][0]['value']
            text.append(f'Caption: {caption}')

        url_value = convert_relative_url(processed_content["imageSources"][idx]["value"])
        text.append(f'Image source: {url_value}')

    return '\n'.join(text)



def get_swp_image(web_part, page_id):
    ''' 
    d1d91016-032f-456d-98a4-721247c305e8
    '''
    text = get_text_from_image(web_part)

    img = {
        'web_part': 'standard_web_part',
        'id': get_composite_id(web_part['id'], page_id),
        'content': 'Image',
        'text': text,
        'linked_to': [{
            'uri': convert_relative_url(web_part['data']['serverProcessedContent']['imageSources'][0]['value'])
        }]
    }


    return img

#### Quick Links

In [49]:
def get_text_from_quickLinks(web_part)->str:
    """
    Process webpart with type `c70391ea-0b10-4ee9-b2b4-006d3fcad0cd`
    """
    text = []
    processed_content = web_part['data']['serverProcessedContent']
    item_properties = web_part['data']['properties']['items']
    searchable_plain_texts = processed_content['searchablePlainTexts']
    links = processed_content['links']

    # get the total amount of links 
    total_links = len(item_properties)

    # Get title
    title = next((f'Title: {item["value"]}' for item in searchable_plain_texts if item['key'] == 'title'), None)
    if title:
        text.append(title)
    
    # Get quick links. 
    for idx in range(total_links):
        title = next((f'Document title: {item["value"]}' for item in searchable_plain_texts 
                      if item['key'] == f'items[{idx}].title'), None)
        link = next((f'Link: {convert_relative_url(item["value"])}' 
                     for item in links if item['key'] == f'items[{idx}].sourceItem.url'), None)
        
        text.append(f'{title}, {link}')

    return '\n'.join(text)


def get_swp_quicklinks(web_part, page_id):
    ''' 
    c70391ea-0b10-4ee9-b2b4-006d3fcad0cd
    TODO merge code from get_Text_from_quickLinks() to here.
    '''
    text = get_text_from_quickLinks(web_part)
    links = web_part['data']['serverProcessedContent']['links']

    linked_to = []
    for idx in range(len(links)):
        link = next((convert_relative_url(item["value"]) for item in links if item['key'] == f'items[{idx}].sourceItem.url'), None)
        if link:
            linked_to.append({'type': get_uri_content_type(link), 'uri': convert_relative_url(link)})

    ql = {
        'web_part': 'standard_web_part',
        'id': get_composite_id(web_part['id'], page_id),
        'content': 'QuickLinks',
        'text': text,
        'linked_to': linked_to
    }
    return ql

#### Title Area

In [50]:
def get_text_from_titleArea(web_part)->str:
    """Get title from Title Area"""
    text = []
    title = web_part['data']['properties']['title']
    authors = web_part['data']['properties']['authors']
    authors_list = [{'name': a['name'], 'email': a['email'] } for a in authors]
    authors_txt = ''
    for author in authors_list:
        authors_txt += f'{author["name"]} ({author["email"]}) '

    text.append(f'Title: {title}')
    text.append(f'Author: {authors_txt}')

    return '\n'.join(text)


def get_swp_titleArea(web_part, page_id):
    """cbe7b0a9-3504-44dd-a3a3-0e5cacd07788"""
    text = get_text_from_titleArea(web_part)
    
    ta = {
        'web_part': 'standard_web_part',
        'id': get_composite_id(web_part['id'], page_id),
        'content': 'TitleArea',
        'text': text
    }

    return ta

#### Link Preview

In [51]:
def get_text_from_linkPreview(web_part)->str:
    """get text from link preview"""
    text = []
    processed_content = web_part['data']['serverProcessedContent']
    # Plain text
    text += [f'{item["key"]}: {item["value"]}' for item in processed_content['searchablePlainTexts']]
    # Link(s)
    text += [f'{item["key"]}: {convert_relative_url(item["value"])}' 
             for item in processed_content['links'] if item['key'] == 'serverRelativeUrl']

    return '\n'.join(text)


def get_swp_linkPreview(web_part, page_id):
    """6410b3b6-d440-4663-8744-378976dc041e"""
    text = get_text_from_linkPreview(web_part)
    url = web_part['data']['serverProcessedContent']['links'][0]['value']

    link_preview = {
        'web_part': 'standard_web_part',
        'id': get_composite_id(web_part['id'], page_id),
        'content': 'LinkPreview',
        'text': text,
        'linked_to': [{'type': get_uri_content_type(url), 'uri': url}]
    }
    return link_preview

#### Get standardWebPart from a webPart component

In [52]:
def get_standardWebPart(wp, page_id)->object:
    """ 
    schema:
    {
        'web_part': 'standard_web_part',
        'content': 'call_to_action|file_and_media|people|image|quick_links',
        'data': [{<data-specific-to-content-type>}]
    }
    """
    swp = {}
    if wp['webPartType'] == 'df8e44e7-edd5-46d5-90da-aca1539313b8':  # call-to-action
        swp = get_swp_call_to_action(wp, page_id)
    elif wp['webPartType'] == 'b7dd04e1-19ce-4b24-9132-b60a1c2b910d':  # File and media
        swp = get_swp_file_and_media(wp, page_id)
    elif wp['webPartType'] == '7f718435-ee4d-431c-bdbf-9c4ff326f46e':  # people
        swp = get_swp_people(wp, page_id)
    elif wp['webPartType'] == 'd1d91016-032f-456d-98a4-721247c305e8':  # Image
        swp = get_swp_image(wp, page_id)
    elif wp['webPartType'] == 'c70391ea-0b10-4ee9-b2b4-006d3fcad0cd':  # Quick links
        swp = get_swp_quicklinks(wp, page_id)
    elif wp['webPartType'] == 'cbe7b0a9-3504-44dd-a3a3-0e5cacd07788':  # Title Area
        swp = get_swp_titleArea(wp, page_id)
    elif wp['webPartType'] == '6410b3b6-d440-4663-8744-378976dc041e':  # Link Preview
        swp = get_swp_linkPreview(wp, page_id)
    else:
        print(f'Unknown `standard web part` type: {wp["webPartType"]}. \nSkipped: {wp}')

    return swp

In [53]:
def add_webpart_order(webparts):
    """
    Add order number to webparts so that it can be sequenced.
    """
    for idx, wp in enumerate(webparts):
        wp['order'] = idx
    return webparts

### Get all WebParts

In [54]:
def get_webparts(canvas_layout, page_id)->list[object]:
    """
    Only processes `horizontalSections`.
    """
    print(f'Working on {page_id}')

    webparts = []

    if 'horizontalSections' in canvas_layout:
        for hs in canvas_layout['horizontalSections']:
            if 'columns' in hs:
                for col in hs['columns']:
                    if 'webparts' in col:
                        for wp in col['webparts']:
                            if wp['@odata.type'] == '#microsoft.graph.textWebPart':
                                twp = get_textWebPart(wp, page_id)
                                if twp:
                                    webparts.append(twp)
                            elif wp['@odata.type'] == '#microsoft.graph.standardWebPart':
                                swp = get_standardWebPart(wp, page_id)
                                if swp:  # skip unknown standard-web-part
                                    webparts.append(swp)
                            else:
                                print(f'Unknown `webpart` type: {wp["@odata.type"]}')
    webparts = add_webpart_order(webparts)
    return webparts

## Load page files

In [55]:
pages = []

for filename in os.listdir(DATA_DIR):
    with open(os.path.join(DATA_DIR, filename), 'r') as f:
        pages.append(json.load(f))

In [56]:
pages_df = pd.DataFrame(pages)

pages_df.head()

,id,@odata.context,canvasLayout@odata.context,name,webUrl,canvasLayout
0,91ce1a95-8b21-4c3c-bb3a-69c0cd294f98,https://graph.microsoft.com/v1.0/$metadata#sit...,https://graph.microsoft.com/v1.0/$metadata#sit...,IoT-Smart-Manufacturing.aspx,https://muarwi.sharepoint.com/sites/XYZKnowled...,{'horizontalSections@odata.context': 'https://...
1,f6a2be01-0fd8-4937-aa78-0df82539e0af,https://graph.microsoft.com/v1.0/$metadata#sit...,https://graph.microsoft.com/v1.0/$metadata#sit...,IoT-Environment.aspx,https://muarwi.sharepoint.com/sites/XYZKnowled...,{'horizontalSections@odata.context': 'https://...
2,fc8157b4-cb7f-482e-8afb-800688fd8ee8,https://graph.microsoft.com/v1.0/$metadata#sit...,https://graph.microsoft.com/v1.0/$metadata#sit...,Home.aspx,https://muarwi.sharepoint.com/sites/XYZKnowled...,{'horizontalSections@odata.context': 'https://...
3,8199d80c-b674-4b9e-8782-b175dbf22437,https://graph.microsoft.com/v1.0/$metadata#sit...,https://graph.microsoft.com/v1.0/$metadata#sit...,IoT-Asset-Performance-Management.aspx,https://muarwi.sharepoint.com/sites/XYZKnowled...,{'horizontalSections@odata.context': 'https://...


In [57]:
pages_df['webParts'] = pages_df.apply(lambda x: get_webparts(x['canvasLayout'], x['id']), axis=1)

Working on 91ce1a95-8b21-4c3c-bb3a-69c0cd294f98
Working on f6a2be01-0fd8-4937-aa78-0df82539e0af
Working on fc8157b4-cb7f-482e-8afb-800688fd8ee8
Working on 8199d80c-b674-4b9e-8782-b175dbf22437


In [58]:
pages_df.head()

,id,@odata.context,canvasLayout@odata.context,name,webUrl,canvasLayout,webParts
0,91ce1a95-8b21-4c3c-bb3a-69c0cd294f98,https://graph.microsoft.com/v1.0/$metadata#sit...,https://graph.microsoft.com/v1.0/$metadata#sit...,IoT-Smart-Manufacturing.aspx,https://muarwi.sharepoint.com/sites/XYZKnowled...,{'horizontalSections@odata.context': 'https://...,"[{'web_part': 'standard_web_part', 'id': '91ce..."
1,f6a2be01-0fd8-4937-aa78-0df82539e0af,https://graph.microsoft.com/v1.0/$metadata#sit...,https://graph.microsoft.com/v1.0/$metadata#sit...,IoT-Environment.aspx,https://muarwi.sharepoint.com/sites/XYZKnowled...,{'horizontalSections@odata.context': 'https://...,"[{'web_part': 'standard_web_part', 'id': 'f6a2..."
2,fc8157b4-cb7f-482e-8afb-800688fd8ee8,https://graph.microsoft.com/v1.0/$metadata#sit...,https://graph.microsoft.com/v1.0/$metadata#sit...,Home.aspx,https://muarwi.sharepoint.com/sites/XYZKnowled...,{'horizontalSections@odata.context': 'https://...,"[{'web_part': 'standard_web_part', 'id': 'fc81..."
3,8199d80c-b674-4b9e-8782-b175dbf22437,https://graph.microsoft.com/v1.0/$metadata#sit...,https://graph.microsoft.com/v1.0/$metadata#sit...,IoT-Asset-Performance-Management.aspx,https://muarwi.sharepoint.com/sites/XYZKnowled...,{'horizontalSections@odata.context': 'https://...,"[{'web_part': 'standard_web_part', 'id': '8199..."


# ------

In [24]:
pages_df.at[0, 'webParts']

[{'web_part': 'standard_web_part',
  'id': '91ce1a95-8b21-4c3c-bb3a-69c0cd294f98#d0282792-5a63-4df7-aafe-f628bcb49576',
  'content': 'TitleArea',
  'text': 'Title: XYZ IoT Manufactr\nAuthor: Muhammad Arif Wicaksana (arif@muarwi.onmicrosoft.com) ',
  'order': 0},
 {'web_part': 'text_web_part',
  'id': '91ce1a95-8b21-4c3c-bb3a-69c0cd294f98#818c65e6-e5a1-4794-bebf-2840abe84a0e',
  'content': 'Text',
  'text': "What is Manufactr?\nXYZ IoT Manufactr provides a one-stop solution for the Smart Manufacturing. This means that customers don't have to worry about connecting various parts of a complex systems. XYZ IoT Manufactr features and applications can be customized to match the business processes, requirements, and integrated into the company's database or backend systems. XYZ IoT Manufactr solution provides customized solutions to meet customer needs. The implementation of this solution begins with a site visit to assess the field conditions in order to provide the right Smart Manufacturing

In [25]:
pages_df.at[1, 'webParts']

[{'web_part': 'standard_web_part',
  'id': 'f6a2be01-0fd8-4937-aa78-0df82539e0af#8540ace7-894c-46e5-82db-e3f9aad26209',
  'content': 'TitleArea',
  'text': 'Title: XYZ IoT Viron\nAuthor: Muhammad Arif Wicaksana (arif@muarwi.onmicrosoft.com) ',
  'order': 0},
 {'web_part': 'standard_web_part',
  'id': 'f6a2be01-0fd8-4937-aa78-0df82539e0af#01c4535e-fc15-4f23-bf15-da89b3a4fb32',
  'content': 'CallToAction',
  'text': 'Overlay text: Contact us for pre-sales support to your prospect\nLink: https://google.com',
  'linked_to': [{'type': 'WebPage', 'uri': 'https://google.com'}],
  'order': 1},
 {'web_part': 'text_web_part',
  'id': 'f6a2be01-0fd8-4937-aa78-0df82539e0af#95c052f1-d429-43e3-9e14-20d8cea540c2',
  'content': 'Text',
  'text': "Introducing XYZ IoT Viron - The Environmental Monitoring Solution\n\xa0\nXYZ IoT Viron provides cutting-edge, real-time insights into your customers' environment—helping them monitor, analyze, and optimize key environmental parameters like temperature, humidi

In [26]:
pages_df.at[2, 'webParts']

[{'web_part': 'standard_web_part',
  'id': 'fc8157b4-cb7f-482e-8afb-800688fd8ee8#1008e4ea-8be4-47d3-afaa-8542e658160e',
  'content': 'TitleArea',
  'text': 'Title: XYZ - Internet of Things Solutions\nAuthor: Muhammad Arif Wicaksana (arif@muarwi.onmicrosoft.com) ',
  'order': 0},
 {'web_part': 'text_web_part',
  'id': 'fc8157b4-cb7f-482e-8afb-800688fd8ee8#24891a22-5552-4462-a81d-b77d7a5bcb37',
  'content': 'Text',
  'text': 'Welcome to the knowledge base of XYZ IoT solutions!',
  'linked_to': [],
  'order': 1},
 {'web_part': 'standard_web_part',
  'id': 'fc8157b4-cb7f-482e-8afb-800688fd8ee8#ccac6748-9dd9-43df-84d0-1ab9466459f7',
  'content': 'LinkPreview',
  'text': 'title: XYZ IOT SOLUTIONS\ndescription: We offer wide range of IoT solutions for your business needs.',
  'linked_to': [{'type': 'WebPage',
    'uri': 'https://xyz-iot-solutions.weebly.com/'}],
  'order': 2},
 {'web_part': 'text_web_part',
  'id': 'fc8157b4-cb7f-482e-8afb-800688fd8ee8#eadb28a0-61a4-4d03-986f-0c0d3de52ff5',
 

In [27]:
pages_df.at[3, 'webParts']

[{'web_part': 'standard_web_part',
  'id': '8199d80c-b674-4b9e-8782-b175dbf22437#e3cd9704-4e84-47a6-af42-f2920ea8164d',
  'content': 'TitleArea',
  'text': 'Title: XYZ IoT Asset Performance \nAuthor: Muhammad Arif Wicaksana (arif@muarwi.onmicrosoft.com) ',
  'order': 0},
 {'web_part': 'text_web_part',
  'id': '8199d80c-b674-4b9e-8782-b175dbf22437#15cdf919-00f0-4c93-8c1c-5102a02338be',
  'content': 'Text',
  'text': 'What is XYZ IoT Asset Performance?\nXYZ IoT Asset Performance is an end-to-end solution to manage and monitor indoors, outdoors, and on the move business assets in real-time. With an automated system of reports, alerts, and insight data retrieval, the entire management process will be easier.\xa0\n\xa0\nAs of now we are specializing to provide asset performance management for vaccine monitoring, ensuring stable temperature during distribution to the end users.\n\xa0\n\xa0\nWhy IoT Asset Performance is the right solution for your business\nTotal visibility: Real-time monitor

# ----

## Connect to Neo4j

In [59]:
gds = GraphDataScience(uri, auth=(username, password))

### Constraints

In [60]:
gds.run_cypher('CREATE CONSTRAINT PageUrlUnique IF NOT EXISTS FOR (p:Page) REQUIRE p.url IS UNIQUE')
gds.run_cypher('CREATE CONSTRAINT WebPartIdUnique IF NOT EXISTS FOR (wp:WebPart) REQUIRE wp.id IS UNIQUE')
gds.run_cypher('CREATE CONSTRAINT PersonIdUnique IF NOT EXISTS FOR (p:Person) REQUIRE p.id IS UNIQUE')
gds.run_cypher('CREATE CONSTRAINT ImageUrlUnique IF NOT EXISTS FOR (i:Image) REQUIRE i.url IS UNIQUE')

""


### Create `Page` and `WebPart` Nodes

`Page` nodes and the respective `WebPart` nodes.

In [61]:
gds.run_cypher(''' 
    UNWIND $data AS row
    CALL {
        WITH row
        MERGE(page:WebPage {url: row.webUrl})
        ON CREATE
            SET page.name = row.name, page.id = row.id
        ON MATCH
            SET page.name = row.name, page.id = row.id

        WITH page, row
        UNWIND row.webParts AS wp
            CREATE(webpart:WebPart)
            SET webpart.id = wp.id
            SET webpart.web_part = wp.web_part
            SET webpart.content = wp.content
            SET webpart.text = wp.text
            SET webpart.order = wp.order
            MERGE (webpart)-[:PART_OF]->(page)
        
        RETURN null AS discard

    } IN TRANSACTIONS OF 2000 ROWS
    FINISH
''',
params={
    'data': pages_df.to_dict(orient='records')
})

""


### Create `PART_OF` relationships

Create relationships between webparts to represent the order in the page.

In [62]:
gds.run_cypher(''' 
    UNWIND $data AS row
    CALL {
        // Create relationships between webparts to represent the order in the page.
        WITH row
        match (wp:WebPart)-[:PART_OF]->(p:WebPage)
        with p, apoc.coll.sortNodes( collect(wp), '^order') as webparts
        call apoc.nodes.link(webparts, 'NEXT', {avoidDuplicates: true})
               
        // Create relationships between pages and the first webpart
        MATCH (wp:WebPart)-[r:PART_OF]->(p:WebPage) WHERE wp.order = 0
        MERGE (p)-[:STARTS_FROM]->(wp)
        

    } IN TRANSACTIONS OF 2000 ROWS
''',
params={
    'data': pages_df.to_dict(orient='records')
})

""


### Create Other Nodes

In [63]:
gds.run_cypher(''' 
    UNWIND $data AS row
    WITH row.webParts AS webparts
    UNWIND webparts AS wp
    UNWIND wp.linked_to AS linked_to 
    WITH wp, linked_to
    CALL apoc.do.case(
        [
            wp.content = 'QuickLinks' AND linked_to.type = 'WebPage', 'MERGE (p:WebPage {url: linked_to.uri})',
            wp.content = 'CallToAction', 'MERGE (p:WebPage {url: linked_to.uri})', 
            wp.content = 'Text' AND linked_to.type = 'Email', 'MERGE (p:Email {uri: linked_to.uri})',
            wp.content = 'Text' AND linked_to.type = 'WebPage', 'MERGE (p:WebPage {url: linked_to.uri})',
            wp.content = 'People', 'MERGE (p:Employee {id: linked_to.id}) ON CREATE SET p.name = linked_to.name, p.email = linked_to.email, p.role = linked_to.role',
            wp.content = 'Image', 'MERGE (p:Image {url: linked_to.uri})',
            wp.content = 'FileAndMedia', 'MERGE (p:FileAndMedia {url: linked_to.uri})',
            wp.content = 'QuickLinks' AND linked_to.type = 'Document', 'MERGE (p:Document {url: linked_to.uri})',
            wp.content = 'QuickLinks' AND linked_to.type = 'WebPage', 'MERGE (p:WebPage {url: linked_to.uri})',
            wp.content = 'LinkPreview', 'MERGE (p:WebPage {url: linked_to.uri})'
        ], 
        'MERGE (p:Other {type: linked_to.type})', {wp: wp, linked_to: linked_to}
    ) YIELD value
    RETURN value
''',
params={
    'data': pages_df.to_dict(orient='records')
})

,value


### Create `LINKED_TO` relationships

In [64]:
gds.run_cypher(''' 
    UNWIND $data AS row
    WITH row.webParts AS webparts
    UNWIND webparts AS wp
    UNWIND wp.linked_to AS linked_to 
    WITH wp, linked_to
    CALL apoc.do.case(
        [
            wp.content = 'People', 'MATCH(webpart:WebPart {id: wp.id}), (p:Employee {id: linked_to.id}) MERGE (webpart)-[:LINKED_TO]->(p)',
            wp.content = 'Image', 'MATCH(webpart:WebPart {id: wp.id}), (i:Image {url: linked_to.uri}) MERGE (webpart)-[:LINKED_TO]->(i)',
            wp.content = 'LinkPreview', 'MATCH(webpart:WebPart {id: wp.id}), (i:WebPage {url: linked_to.uri}) MERGE (webpart)-[:LINKED_TO]->(i)',
            wp.content = 'Text' AND linked_to.type = 'WebPage', 'MATCH(webpart:WebPart {id: wp.id}), (i:WebPage {url: linked_to.uri}) MERGE (webpart)-[:LINKED_TO]->(i)',
            wp.content = 'QuickLinks' AND linked_to.type = 'Document', 'MATCH(webpart:WebPart {id: wp.id}), (d:Document {url: linked_to.uri}) MERGE (webpart)-[:LINKED_TO]->(d)',
            wp.content = 'QuickLinks' AND linked_to.type = 'WebPage', 'MATCH(webpart:WebPart {id: wp.id}), (w:WebPage {url: linked_to.uri}) MERGE (webpart)-[:LINKED_TO]->(w)',
            wp.content = 'FileAndMedia', 'MATCH(webpart:WebPart {id: wp.id}), (fm:FileAndMedia {url: linked_to.uri}) MERGE (webpart)-[:LINKED_TO]->(fm)',
            wp.content = 'CallToAction', 'MATCH(webpart:WebPart {id: wp.id}), (cta:WebPage {url: linked_to.uri}) MERGE (webpart)-[:LINKED_TO]->(cta)'
        ],
        'MATCH (webpart:WebPart {id: wp.id}), (linked {url: linked_to.url}) MERGE (webpart)-[:LINKED_TO]->(linked)',
        {wp:wp, linked_to: linked_to}
    ) YIELD value
    RETURN value
''',
params={
    'data': pages_df.to_dict(orient='records')
})

,value
